In [9]:
import pandas as pd
import numpy as np
import os 

#input for any file name
file_name=input("Enter File Name: ")
journey=input("Enter 'up' or 'down': ").lower()
file_path=os.path.join('week_data',journey, file_name)

if not os.path.isfile(file_path):
    print(f"Error: File '{file_name}' does not exist.")
else:
    df=pd.read_csv(file_path)

Enter File Name: 04-02-19.csv
Enter 'up' or 'down': up


In [10]:
#Only 519 buses
bus_no = int(input("Enter bus Number: "))
df = df[df['Schedule Name'].str.contains(rf'\b{bus_no}\b')]

Enter bus Number: 519


In [11]:
cols=['Schedule Name','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]

['04-02-2019' '01-02-2019' '03-02-2019' '04-01-2019']
['04-02-2019' '01-02-2019' '03-02-2019' '04-01-2019']


In [ ]:
source_stage=input("Enter source: ")
destination_stage=input("Enter destination: ")
df=df[(df['Source']==source_stage) & (df['Destination']==destination_stage)]

# Convert the trip start time and end time columns to datetime objects
df['Trip Start Time'] = pd.to_datetime(df['Trip Start Time'], format='%H:%M:%S')
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'], format='%H:%M:%S')

print("\t\t\t\tBoarding Time\t\t\t\t")
# values of start time and end time 
start=input("Enter Start time: ")
end=input("Enter End time: ")
start_time = pd.to_datetime(start).time()
end_time = pd.to_datetime(end).time()

df = df[(df['Trip Start Time'].dt.time >= start_time) & (df['Trip Start Time'].dt.time < end_time)]

In [ ]:
bus_stages = [
    'T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]

#assigning numbers from 0 to length of bus stages
stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)
df.dropna(subset=['From Stage', 'To Stage'], inplace=True)

# Create an empty OD matrix with stage names as indexes and columns
od_matrix = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

# Populate the matrix 
for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    if not pd.isna(adult_count):
        od_matrix.loc[source, destination] += adult_count
    else:
        pass

od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING', :] = od_matrix.sum(axis=0)

od_matrix


In [ ]:
store_file=input("Enter store file ")
od_matrix.to_excel(f"{store_file}.xlsx")